In [1]:
import jieba
import sys
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfVectorizer
from datetime import timedelta, date

In [2]:
def is_chinese(uchar):         
    if u'\u4e00' <= uchar<=u'\u9fff':
        return True
    else:
        return False

In [3]:
#專有名詞
with open('ProperN.txt', 'rt',encoding="utf-8") as txt:
    ProperN= txt.read()
ProperN = ProperN.split("\n")
for w in ProperN:
    jieba.add_word(w)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/bf/kxrhqhkd3s1263kmtf84x8zw0000gn/T/jieba.cache
Loading model cost 0.710 seconds.
Prefix dict has been built succesfully.


In [4]:
#停用詞
with open('StopWords.txt', 'rt',encoding="utf-8") as txt:
    stopword = txt.read()
stopword = stopword.split("\n")

In [5]:
def textMining(file):
    font = r"/anaconda3/lib/python3.6/site-packages/matplotlib/mpl-data/fonts/ttf/SimHei.ttf"
    #open file
    with open("./News/" + file + ".txt", 'rt',encoding="utf-8") as txt:
        data = txt.read()
    #text segmentation
    seg_list = jieba.cut(data, cut_all=False)
    seg_list = list(seg_list)
    filter_list = []
    wordDict = {}
    jump = False
    for word in seg_list:
        for s in word:
            if not is_chinese(s):
                jump = True
                break
        if not jump:
            filter_list.append(word)
        jump = False
    return filter_list

In [6]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

start_date = date(2017, 1, 6)
end_date = date(2018, 10, 1)
dates = []
for single_date in daterange(start_date, end_date):
    dates.append(single_date.strftime("%Y%m%d"))

In [7]:
def get_join_list(file_date):
    try:
        filter_list = textMining(file_date)
        join_list = " ".join(filter_list)
        return join_list
    except:
        print("no news on " + file_date)

In [8]:
corpus = []
news_dates = []
for date in dates:
    news = get_join_list(date)
    if news != None:
        print("append news of date " + date)
        news_dates.append(date)
        corpus.append(news)

append news of date 20170106
no news on 20170107
no news on 20170108
append news of date 20170109
no news on 20170110
no news on 20170111
no news on 20170112
no news on 20170113
no news on 20170114
no news on 20170115
no news on 20170116
no news on 20170117
no news on 20170118
no news on 20170119
no news on 20170120
no news on 20170121
no news on 20170122
no news on 20170123
no news on 20170124
append news of date 20170125
append news of date 20170126
append news of date 20170127
no news on 20170128
no news on 20170129
no news on 20170130
no news on 20170131
no news on 20170201
no news on 20170202
no news on 20170203
no news on 20170204
no news on 20170205
no news on 20170206
no news on 20170207
no news on 20170208
no news on 20170209
no news on 20170210
no news on 20170211
no news on 20170212
no news on 20170213
no news on 20170214
no news on 20170215
no news on 20170216
append news of date 20170217
no news on 20170218
no news on 20170219
no news on 20170220
append news of date 201702

append news of date 20180425
no news on 20180426
no news on 20180427
no news on 20180428
no news on 20180429
no news on 20180430
no news on 20180501
no news on 20180502
no news on 20180503
append news of date 20180504
no news on 20180505
no news on 20180506
no news on 20180507
no news on 20180508
no news on 20180509
no news on 20180510
no news on 20180511
no news on 20180512
no news on 20180513
append news of date 20180514
append news of date 20180515
append news of date 20180516
no news on 20180517
no news on 20180518
no news on 20180519
no news on 20180520
append news of date 20180521
no news on 20180522
no news on 20180523
no news on 20180524
append news of date 20180525
no news on 20180526
no news on 20180527
append news of date 20180528
append news of date 20180529
append news of date 20180530
no news on 20180531
no news on 20180601
no news on 20180602
no news on 20180603
no news on 20180604
no news on 20180605
no news on 20180606
no news on 20180607
no news on 20180608
no news on

In [9]:
len(corpus)

97

In [23]:
# tfidf
vectorizer = TfidfVectorizer(max_df = 0.9, min_df = 0.2)
tfidf = vectorizer.fit_transform(corpus)

In [24]:
words = vectorizer.get_feature_names()
print("len(words): ", len(words))

len(words):  111


In [25]:
weight = tfidf.T.toarray()
len(weight)

111

In [26]:
print("tfidf.shape: ", tfidf.shape)

tfidf.shape:  (97, 111)


<97x111 sparse matrix of type '<class 'numpy.float64'>'
	with 3402 stored elements in Compressed Sparse Row format>

In [27]:
# check each document
for i in range(len(corpus)):
    print('----{0} NEWS----'.format(news_dates[i]))
    for j in range(len(words)):
        if tfidf[i,j] > 0.2:
              print(words[j], tfidf[i,j], i ,j)

----20170106 NEWS----
企業 0.22028073672032122 0 6
增加 0.23112747343071707 0 27
東元電機 0.4491978767136803 0 65
邱純枝 0.2848750669045738 0 99
馬達 0.21137148139215298 0 110
----20170109 NEWS----
技術 0.20392312600613358 1 43
推出 0.5089434669819464 1 49
智慧 0.2152545860608882 1 60
目前 0.2123078428752135 1 74
研發 0.21828434692588086 1 78
空調 0.26343937535467205 1 81
管理系 0.25886211443561424 1 82
能源 0.23133663406370003 1 88
銷售 0.26343937535467205 1 101
----20170125 NEWS----
今年 0.40314199938085193 2 3
以上 0.290868494907465 2 4
優勢 0.44164335042975605 2 9
去年 0.35592381330120937 2 15
同時 0.22082167521487803 2 22
發展 0.266942859975907 2 73
集團 0.2812008411947529 2 105
----20170126 NEWS----
今年 0.5034742022659956 3 3
以上 0.290606850514722 3 4
億元 0.2929771989917212 3 8
公司 0.22072323127666577 3 11
其中 0.22826860353655168 3 12
去年 0.4667297903883427 3 15
開始 0.24388842724214127 3 102
----20170127 NEWS----
台灣 0.24615318809953685 4 19
推動 0.2592734951204332 4 50
智慧 0.7353519481271065 4 60
產業 0.26738800090109976 4 72
發展 0.20146

馬達 0.43728783957572814 69 110
----20180329 NEWS----
台灣 0.2596995303553421 70 19
國際 0.407629460140101 70 25
工業 0.370570145298119 70 32
市場 0.5079126865671938 70 33
推動 0.43766700199153763 70 50
產業 0.32240336094443883 70 72
馬達 0.2749554793351106 70 110
----20180421 NEWS----
今年 0.2347339236492755 71 3
可望 0.34388638304318236 71 17
合併營收 0.36642100406278205 71 21
市場 0.2321309931164244 71 33
空調 0.38572764680634963 71 81
----20180423 NEWS----
億元 0.35744217242182064 72 8
全球 0.5655403204234204 72 10
提供 0.23829478161454706 72 51
智能 0.3364595390953806 72 61
營收 0.31443487976872697 72 69
產品 0.24124887301305356 72 71
----20180425 NEWS----
事業 0.3117545554486351 73 2
布局 0.29536804864564803 73 34
目前 0.24224847810943775 73 74
董事長 0.5091972346467243 73 90
邱純枝 0.5279215951134693 73 99
集團 0.3189843503805965 73 105
----20180504 NEWS----
中心 0.32595033833786236 74 0
企業 0.8431559962660944 74 6
公司 0.3105698684877249 74 11
----20180514 NEWS----
億元 0.8108321355262176 75 8
合併營收 0.3625163547633911 75 21
增加 0.362516354

In [30]:
import pandas as pd
df = pd.DataFrame(weight, index = words, columns = news_dates)
df

,20170106,20170109,20170125,20170126,20170127,20170217,20170221,20170321,20170328,20170415,...,20180705,20180709,20180712,20180718,20180801,20180813,20180822,20180911,20180927,20180928
中心,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.136721,0.000000,0.000000,...,0.219495,0.000000,0.000000,0.000000,0.000000,0.000000,0.164345,0.000000,0.000000,0.000000
主要,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.068680,0.000000,...,0.060624,0.092004,0.000000,0.000000,0.083623,0.000000,0.000000,0.043291,0.000000,0.000000
事業,0.168228,0.000000,0.000000,0.085806,0.000000,0.000000,0.000000,0.141798,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
今年,0.197418,0.160316,0.403142,0.503474,0.000000,0.097769,0.142041,0.083201,0.050441,0.000000,...,0.000000,0.000000,0.000000,0.465645,0.030708,0.000000,0.000000,0.095384,0.000000,0.049819
以上,0.000000,0.000000,0.290868,0.290607,0.000000,0.141082,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.097505,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
以及,0.000000,0.121142,0.000000,0.000000,0.091965,0.443273,0.000000,0.000000,0.000000,0.000000,...,0.067289,0.000000,0.000000,0.000000,0.046408,0.000000,0.000000,0.048051,0.135598,0.075291
企業,0.220281,0.000000,0.000000,0.074904,0.000000,0.436369,0.052830,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.139844,0.000000,0.068528,0.000000,0.000000,0.000000,0.000000,0.000000
使用,0.000000,0.131720,0.000000,0.000000,0.000000,0.000000,0.058352,0.136721,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
億元,0.172319,0.000000,0.000000,0.292977,0.070821,0.113786,0.123983,0.000000,0.058704,0.000000,...,0.000000,0.000000,0.000000,0.270964,0.000000,0.860888,0.116396,0.481042,0.000000,0.000000
優勢,0.000000,0.000000,0.441643,0.000000,0.049998,0.000000,0.000000,0.136721,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
